# Coursera Capstone Project

### Luigi De Marco, 10/5/2020

This Jupyter Notebook will be mainly used to complete the Coursera Capstone project in the IBM Data Science Series

In [16]:
import pandas as pd
import numpy as np

In [17]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!
